In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.impute import SimpleImputer
import dask
import dask.dataframe as dd
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
from dask.distributed import Client, progress

In [4]:
client = Client(n_workers=2, memory_limit='3GB')
client

Client Scheduler: tcp://127.0.0.1:49981 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 6.00 GB


__Загружаем датасет feats через dask__

In [5]:
feats_raw = dd.read_csv('E:/IV четверть/3. Видеокурс Megafon + курсовой проект/Данный курсового проекта/features.csv', 
                    delim_whitespace=True)

In [6]:
feats_raw.head(1)

,id,buy_time,0,1,2,3,4,5,6,7,...,243,244,245,246,247,248,249,250,251,252
0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,-65.076097,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [7]:
feats_raw.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 255 entries, id to 252
dtypes: float64(253), int64(2)

__Загружаем train__ 

In [8]:
train = pd.read_csv('E:/IV четверть/3. Видеокурс Megafon + курсовой проект/Данный курсового проекта/data_train.csv')

In [9]:
train.head(1)

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0


__Переведем признак buy_time из timestamp в datetime__

In [10]:
train['buy_time'] = pd.to_datetime(train['buy_time'], unit='s')

__Загрузим test и проделаем те же операции что и с train__

In [11]:
test = pd.read_csv('E:/IV четверть/3. Видеокурс Megafon + курсовой проект/Данный курсового проекта/data_test.csv')

In [12]:
test['buy_time'] = pd.to_datetime(test['buy_time'], unit='s')

__Сохраним в отдельную переменную датафреймы из features_raw в которых есть только соответствующие id__

In [13]:
feats_train = feats_raw[feats_raw['id'].isin(train['id'])].compute()

In [14]:
feats_test = feats_raw[feats_raw['id'].isin(test['id'])].compute()

__Проверим даты по которым есть дубликаты id в feats_train и feats_test, предварительно переведя даты в datetime__

! Удалим дубликаты по id путем сортировки по id и по buy_time и дальнейшем удалении дубликата с датой, которая дальше от даты покупки из train и test соответственно.

__---train---__

In [15]:
feats_train['buy_time'] = pd.to_datetime(feats_train['buy_time'], unit='s')

In [16]:
feats_train = feats_train.sort_values(by=['id', 'buy_time']).drop_duplicates(subset=['id'], keep='last')

In [17]:
feats_train.id.duplicated().value_counts()

False    806613
Name: id, dtype: int64

In [18]:
feats_train = feats_train.rename(columns={'buy_time':'buy_time_2'})

__---test---__

In [19]:
feats_test['buy_time'] = pd.to_datetime(feats_test['buy_time'], unit='s')

In [20]:
feats_test = feats_test.sort_values(by=['id', 'buy_time']).drop_duplicates(subset=['id'], keep='last')

In [21]:
feats_test.id.duplicated().value_counts()

False    70152
Name: id, dtype: int64

In [22]:
feats_test = feats_test.rename(columns={'buy_time':'buy_time_2'})

__Соединим train и test с feats_train и feats_test соответственно__

In [23]:
train_df = train.merge(feats_train, on=['id'], how='outer')

In [24]:
test_df = test.merge(feats_test, on=['id'], how='outer')

__Добавим колонку с временным лагом между датами в днях в абсолютном значении__

__---train---__

In [25]:
train_df['dates_lag'] = (train_df.buy_time - train_df.buy_time_2).dt.days

In [26]:
train_df.dates_lag = train_df.dates_lag.astype('int8')

In [27]:
train_df.dates_lag = train_df.dates_lag.abs()

__---test---__

In [28]:
test_df['dates_lag'] = (test_df.buy_time - test_df.buy_time_2).dt.days

In [29]:
test_df.dates_lag = test_df.dates_lag.astype('int8')

In [30]:
test_df.dates_lag = test_df.dates_lag.abs()

__Добавим некоторые колонки сформированные из обеих дат для test и train__

__---train---__

__!TODO Добавить праздничный ли это день. Возможно предыдущий или следующий праздничные__

In [31]:
import holidays
from datetime import date

In [32]:
def feats_from_date(df):
    
    df['week_day'] = pd.DatetimeIndex(df['buy_time']).weekday
    df['week_day_2'] = pd.DatetimeIndex(df['buy_time_2']).weekday

    df['month_day'] = pd.DatetimeIndex(df['buy_time']).day
    df['month_day_2'] = pd.DatetimeIndex(df['buy_time_2']).day

    df['year_day'] = pd.DatetimeIndex(df['buy_time']).dayofyear
    df['year_day_2'] = pd.DatetimeIndex(df['buy_time_2']).dayofyear

    df['year_week'] = pd.DatetimeIndex(df['buy_time']).week
    df['year_week_2'] = pd.DatetimeIndex(df['buy_time_2']).week

    df['year_month'] = pd.DatetimeIndex(df['buy_time']).month
    df['year_month_2'] = pd.DatetimeIndex(df['buy_time_2']).month

    df['year'] = pd.DatetimeIndex(df['buy_time']).year
    df['year_2'] = pd.DatetimeIndex(df['buy_time_2']).year

    df['quart'] = pd.DatetimeIndex(df['buy_time']).quarter
    df['quart_2'] = pd.DatetimeIndex(df['buy_time_2']).quarter

    df = df.astype({'week_day':'int8',
                    'week_day_2':'int8',
                    'month_day':'int8',
                    'month_day_2':'int8',
                    'year_day':'int8',
                    'year_day_2':'int8',
                    'year_week':'int8',
                    'year_week_2':'int8',
                    'year_month':'int8',
                    'year_month_2':'int8',
                    'year':'int8',
                    'year_2':'int8',
                    'quart':'int8',
                    'quart_2':'int8'})
    
    return df

In [33]:
train_df = feats_from_date(train_df)

In [34]:
train_df.target = train_df.target.astype('int8')

In [35]:
test_df = feats_from_date(test_df)

__На данный момент train_df занимает около 4Гб - это много! Переведем типы данных из 64-битных в 32-битные__

In [38]:
train_df.select_dtypes('float64').columns.isin(test_df.select_dtypes('float64').columns).any()

True

In [39]:
train_df.select_dtypes('int64').columns.isin(test_df.select_dtypes('int64').columns).any()

True

In [40]:
def dtyper(df):

    float64_cols = df.select_dtypes('float64').columns
    int64_cols = df.select_dtypes('int64').columns

    for col in float64_cols:
        df[f'{col}'] = df[f'{col}'].astype('float32')

    for col in int64_cols:
        df[f'{col}'] = df[f'{col}'].astype('int32')
        
    print(df.dtypes.value_counts())

In [41]:
dtyper(train_df)

float32           254
int8               16
int32               2
datetime64[ns]      2
dtype: int64


In [42]:
dtyper(test_df)

float32           254
int8               15
int32               2
datetime64[ns]      2
dtype: int64


!если будет низки скор f1 то попробовать прогноз вероятности классов с занижением порога отсева

In [6]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score

__Сбросим даты, тк мы вытащили всю ужную информацию из них.__

In [7]:
def dates_drop(df):
    df = df.drop(columns=['buy_time', 'buy_time_2'])
    
    return df

In [8]:
train_df = dates_drop(train_df)

In [9]:
test_df = dates_drop(test_df)

__Тестовая выборка составляет примерно 1/10 от тренировоной - таким будет размер валидационной выборки = 0.1 от трейна__

In [10]:
len(test_df)/len(train_df)

0.085649904467368

In [11]:
X = train_df.loc[:, train_df.columns != 'target']

In [12]:
y = train_df['target']

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, test_size=0.1, stratify=y)

__Проверим стратифицированность тренировочной и валидационной выборок по целевому классу__

In [14]:
y_train.value_counts()[1] / y_train.value_counts()[0]

0.07801446019126627

In [15]:
y_valid.value_counts()[1] / y_valid.value_counts()[0]

0.07801988411733445

!Пропорции соблюдены